### Importación de datos



In [ ]:
import polars as pl
"""Decidí utilizar polars en lugar de pandas a fin de optimizar el tiempo de respuesta"""

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda1 = pl.read_csv(url)
tienda2 = pl.read_csv(url2)
tienda3 = pl.read_csv(url3)
tienda4 = pl.read_csv(url4)

tienda1.head()

# 1. Análisis de facturación



In [ ]:
""" Usando funciones nativas """
ingresos = [sum(t['Precio']) for t in [tienda1, tienda2, tienda3, tienda4]]
print(ingresos)

In [ ]:
""" Forma alternativa directamente en el df """
ingresos = [df["Precio"].sum() for df in [tienda1, tienda2, tienda3, tienda4]]
print(ingresos)

# 2. Ventas por categoría

In [ ]:
from typing import List, Tuple

""" Usando funciones nativas """
def ventas_tienda(tiendas: List[pl.DataFrame]) -> List[Tuple]:
    categorias = set(cat for tienda in tiendas for cat in tienda["Categoría del Producto"].to_list())
    resultados = [["Categoría"] + [f"Tienda_{i}" for i in range(1, len(tiendas)+1)]]
    resultados += [[cat] + [tienda["Categoría del Producto"].to_list().count(cat) for tienda in tiendas] for cat in categorias]
    return resultados
    
print(ventas_tienda([tienda1, tienda2, tienda3, tienda4]))

In [ ]:
""" Forma alternativa directamente con DF """
from typing import List
def ventas_tienda(tiendas: List[pl.DataFrame]) -> pl.DataFrame:
    resultados = []
    resultados_top = []
    for i, tienda in enumerate(tiendas, 1):
        df = tienda.group_by("Categoría del Producto").agg([pl.len().alias(f"Tienda {i}")])
        resultados.append(df)
        resultados_top.append(df.sort(f"Tienda {i}", descending=True).head(3))
    resultados = pl.concat(resultados, how="align")
    resultados_top = pl.concat(resultados_top, how ="align")
    return resultados,resultados_top

print(ventas_tienda([tienda1, tienda2, tienda3, tienda4]))

# 3. Calificación promedio de la tienda


In [ ]:
""" Usando funciones nativas """
from typing import List
def promedio(tienda:List[pl.DataFrame] ) -> List[float]:
    calificacion=tienda["Calificación"].to_list()
    return round(sum(calificacion)/len(calificacion),2)
print([promedio(tienda1),promedio(tienda2),promedio(tienda3),promedio(tienda4)])

In [ ]:
""" Forma alternativa directamente con DF """
from typing import List
def promedio(tienda: pl.DataFrame) ->  List[float]:
    return round(tienda["Calificación"].mean(), 2)

print([promedio(t) for t in [tienda1, tienda2, tienda3, tienda4]])

# 4. Productos más y menos vendidos

In [ ]:
""" Usando funciones nativas """
from typing import List
def productos(tiendas:List[pl.DataFrame] ) -> str:
    productos = set(producto for tienda in tiendas for producto in tienda["Producto"].to_list())
    data = [[producto] + [tienda["Producto"].to_list().count(producto) for tienda in tiendas] for producto in productos]
    resumen = []
    for i in range(len(tiendas)):
        tienda_id = f"Tienda_{i+1}"
        productos_ordenados = sorted(data, key=lambda x: (-x[i+1],x[0]))
        max = productos_ordenados[0]
        min = productos_ordenados[-1]
        resumen.append(f"{tienda_id}: más vendido : {max[0]} ({max[i+1]} ventas), menos vendido : {min[0]} ({min[i+1]} ventas)")
    resultados = "Los productos más y menos vendidos por tienda son:\n" + "\n".join(resumen)
    return resultados

print(productos([tienda1, tienda2, tienda3, tienda4]))

In [ ]:
""" Forma alternativa directamente con DF """
from typing import List
def productos(tiendas: List[pl.DataFrame]) -> str:
    resultados = []
    for i, tienda in enumerate(tiendas, 1):
        df = tienda.group_by("Producto").agg([pl.len().alias(f"Tienda {i}")]).sort(by=[f"Tienda {i}", "Producto"], descending=[True, False])
        resultados.append(f"Tienda {i} - más vendido: {df[0]["Producto"][0]} ({df[0][f"Tienda {i}"][0]} ventas), menos vendido: {df[-1]["Producto"][0]} ({df[-1][f"Tienda {i}"][0]} ventas)")
    return "\n".join(resultados)

print(productos([tienda1,tienda2,tienda3,tienda4]))


# 5. Envío promedio por tienda

In [ ]:
""" Usando funciones nativas """
from typing import List
def promedio(tienda:pl.DataFrame ) -> List[float]:
    envio = tienda["Costo de envío"].to_list()
    return round(sum(envio)/len(envio),2)
print([promedio(tienda1),promedio(tienda2),promedio(tienda3),promedio(tienda4)])

In [ ]:
""" Forma alternativa directamente con DF """
def promedio(tienda: pl.DataFrame) ->  List[float]:
    return round(tienda["Costo de envío"].mean(), 2)

print([promedio(t) for t in [tienda1, tienda2, tienda3, tienda4]])

# 6. Generando Gráfico


In [ ]:
""" Gráfico análisis de facturación """
import plotly.express as px
from typing import List
ingresos = [df["Precio"].sum() for df in [tienda1, tienda2, tienda3, tienda4]]
tiendas = ["Tienda 1", "Tienda 2", "Tienda 3", "Tienda 4"]
df_ingresos = pl.DataFrame({
    "Tiendas": tiendas,
    "Ingresos": ingresos
})
fig = px.scatter(
    df_ingresos,
    x='Tiendas',
    y='Ingresos',
    color='Tiendas',
    title="Ingresos por Tienda",
    labels={'Tiendas': '', 'Ingresos': 'Ingresos'},
    hover_data=['Tiendas', 'Ingresos'],
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=[0, 1, 2, 3], ticktext=[]),
    showlegend=False, 
)
fig.show()

In [ ]:
import plotly.express as px
from typing import List
def ventas_tienda(tiendas: List[pl.DataFrame]) -> pl.DataFrame:
    resultados = []
    for i, tienda in enumerate(tiendas, 1):
        df = tienda.group_by("Categoría del Producto").agg([pl.len().alias(f"Tienda {i}")])
        resultados.append(df)
    resultados = pl.concat(resultados, how="align")
    return resultados

df_ventas = ventas_tienda([tienda1, tienda2, tienda3, tienda4])

df_ventas_pd = df_ventas.unpivot(
    index="Categoría del Producto",
    variable_name="Tienda",
    value_name="Cantidad Vendida"
)

fig = px.bar(
    df_ventas_pd,
    x="Tienda",
    y="Cantidad Vendida",
    color="Tienda",
    facet_col="Categoría del Producto",
    facet_col_wrap=4, 
    facet_row_spacing=0.1, 
    title="Cantidad Vendida por Categoría",
)

fig.update_layout(
    height=400,
    showlegend=False,
    margin=dict(t=50, b=30),
)
fig.for_each_xaxis(lambda x: x.update(title=None, showticklabels=False))
fig.for_each_yaxis(lambda y: y.update(title=None, showticklabels=False))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


fig.show()

In [ ]:
""" Gráfico ventas por categoría """
import plotly.express as px
from typing import List
def promedio(tienda: pl.DataFrame) ->  List[float]:
    return round(tienda["Calificación"].mean(), 2)

calificacion= [promedio(t) for t in [tienda1, tienda2, tienda3, tienda4]]
tiendas = ["Tienda 1", "Tienda 2", "Tienda 3", "Tienda 4"]
df_Calificacion = pl.DataFrame({
    "Tiendas": tiendas,
    "Calificacion": calificacion
})
fig = px.scatter(
    df_Calificacion,
    x='Tiendas',
    y='Calificacion',
    color='Tiendas',
    title="Calificación por Tienda",
    labels={'Tiendas': '', 'Calificacion': 'Promedio de Calificación'},
    hover_data=['Tiendas', 'Calificacion'],
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=[0, 1, 2, 3], ticktext=[]),
    showlegend=False, 
)
fig.show()


# 7. Análisis final

Con base en los análisis realizados y los gráficos generados, es momento de sintetizar sus hallazgos en un informe final. Dentro de tu Colab, deberás redactar un texto explicando a qué tienda debe vender el Sr. Juan, teniendo en cuenta todos los factores analizados, como:

Los ingresos totales de las tiendas.

Las categorías de productos más y menos vendidas.

Las calificaciones promedio de los clientes por tienda.

Los productos más y menos vendidos.

El coste de envío promedio para cada tienda.

El propósito del análisis es mediante los datos presentados mostrar cuales son las fortalezas y debilidades de cada tienda, hablando de la primera tienda ha generado una venta de $1,150,880,400.00 (Un mil ciento cincuenta millones ochocientos ochenta mil cuatrocientos pesos) lo que la posiciona en el primer lugar, repuntando en el área de Muebles y Electrónicos, sin embargo tiene una calificacion promedio de 3.98 lo que la posciona en el último lugar.
Hablando de la segunda tienda ha generado una venta de $1,116,343,500.00 (Un mil ciento dieciseis millones trescientos cuarenta y tres mil quinientos pesos) lo que la posiciona en el segundo lugar, repuntando en el área de Muebles y Electrónicos, sin embargo tiene una calificacion promedio de 4.04 lo que la posciona en el segundo lugar.
Hablando de la tercera tienda ha generado una venta de $1,098,019,600.00 (Un mil noventa y ocho millones diecinueve mil seiscientos pesos) lo que la posiciona en el tercer lugar, repuntando en el área de Muebles y Electrónicos, sin embargo tiene una calificacion promedio de 4.05 lo que la posciona en el primer lugar.
Hablando de la cuarta tienda ha generado una venta de $1,038,375,700.00 (Un mil treinta y ocho millones trescientos setenta y cinco mil setecientos pesos) lo que la posiciona en el último lugar, repuntando en el área de Muebles y Electrónicos, sin embargo tiene una calificacion promedio de 4.00 lo que la posciona en el tercer lugar.

Despues de todo el análisis podemos concluir que el departamento que más vende en todas las tiendas es el de Muebles y como segundo lugar el de Electrónicos, y que la tienda 4 no es tan viable mantenerla ya que aunque tiene una calificación de 4.00 lo que la posiciona en el tercer lugar, sin embargo lo que genera en ventas la posiciona en último lugar, y en el caso de la primera sigue siendo mas redituable aunque su calificación sea la mas baja de 3.98.

# 8. Extra
Esta actividad es un extra, por lo tanto es OPCIONAL.

En este extra, tendrás el desafío de explorar las coordenadas geográficas de los datos de ventas e identificar patrones relacionados con la ubicación de las compras. Al utilizar las columnas de latitud y longitud, puede generar visualizaciones para comprender cómo varían las ventas según la ubicación geográfica.

Desafío:

Utilice los datos de latitud (lat) y longitud (lon) para mapear las ventas de cada tienda y analizar la distribución geográfica de los productos vendidos.

Genere gráficos de dispersión o mapas de calor (Heatmaps) para visualizar datos e identificar áreas con la mayor concentración de ventas.

Explore si algunas tiendas tienen un rendimiento superior o inferior al esperado en determinadas regiones e identifique si existen patrones geográficos que puedan influir en el rendimiento de las tiendas.

In [ ]:
import plotly.graph_objects as go
tiendas = [tienda1, tienda2, tienda3, tienda4]
colores_tienda = ['red', 'blue', 'green', 'purple']

fig = go.Figure()

for i, tienda in enumerate(tiendas):
    fig.add_trace(go.Scattergeo(
        lat=tienda['lat'], 
        lon=tienda['lon'], 
        text=tienda['Producto'], 
        marker=dict(size=10, color=colores_tienda[i]),
        name = f"Tienda {i+1}"
    ))
fig.update_layout(
    title='Mapa de Tiendas',
    geo=dict(
        scope='world',
        projection_type='natural earth',
        showland=True,
        landcolor='lightgray',
        subunitcolor='white'
    )
)

fig.show()

In [ ]:
import plotly.express as px
tiendas = [tienda1, tienda2, tienda3, tienda4]

df_tiendas = pl.concat(tiendas)
lats = df_tiendas['lat'].to_list()
lons = df_tiendas['lon'].to_list()
fig = px.density_map(
    lat=lats,
    lon=lons,
    z=[1]*len(df_tiendas),
    radius=10,
    zoom=1,
    title="Mapa de Calor de Tiendas"
)
fig.update_layout(
    autosize=True,
    margin=dict(l=10, r=10, t=40, b=10)
)

fig.show()